In [1]:
import sys
sys.path.insert(0, '..')

In [2]:
import pandas as pd
from pyphoon.db.pd_manager import PDManager
from pyphoon.io.h5 import read_source_image

In [3]:
man = PDManager()
orig_images_dir = '../../../../datasets/typhoon/wnp/image/'
corrected_dir = '../../../database/corrected'
db_dir = '../../../database/'
from os.path import exists, join
images = join(db_dir, 'images.pkl')
corrected = join(db_dir, 'corrected.pkl')

In [4]:
man.load_original_images(images)
man.load_corrected_images(corrected)
man.load_besttrack(join(db_dir, 'besttrack.pkl'))
man.load_missing_images_info(join(db_dir, 'missing.pkl'))

In [5]:
images = man.images
besttrack = man.besttrack
corrected = man.corrected
missing = man.missing

In [6]:
man.missing.head()

,start_time,time_step,frames_num,missing_num,completeness,missing_frames,have_good_neighbours
seq_no,,,,,,,
197830,1978-12-01 00:00:00,03:00:00,3,0,1.000000,[],[]
197901,1978-12-31 12:00:00,03:00:00,147,34,0.768707,"[1, 2, 9, 10, 17, 18, 25, 26, 31, 32, 33, 36, ...","[36, 42, 81]"
197902,1979-03-19 00:00:00,03:00:00,51,6,0.882353,"[5, 12, 19, 26, 33, 40]","[5, 12, 19, 26, 33, 40]"
197903,1979-04-11 00:00:00,03:00:00,101,18,0.821782,"[5, 12, 19, 20, 27, 28, 35, 36, 43, 44, 51, 52...","[5, 12]"
197904,1979-05-09 00:00:00,03:00:00,85,18,0.788235,"[1, 4, 5, 8, 11, 12, 19, 20, 27, 28, 35, 36, 4...","[1, 8]"


In [7]:
one_hour_seqs = missing[missing.time_step == pd.Timedelta(hours=1)]

In [8]:
# getting triplets
seq_no = one_hour_seqs.sample(n=1, random_state=5).index[0]
seq_no

200115

In [9]:
seq = one_hour_seqs.loc[seq_no]
triplets = []
for frame in range(1, seq.frames_num - 1):
    triplet = [frame - 1, frame, frame + 1]
    if len(set(seq.missing_frames).intersection(triplet)) == 0:
        triplets.append(triplet)
triplets

[[0, 1, 2],
 [1, 2, 3],
 [2, 3, 4],
 [3, 4, 5],
 [4, 5, 6],
 [5, 6, 7],
 [6, 7, 8],
 [7, 8, 9],
 [8, 9, 10],
 [9, 10, 11],
 [10, 11, 12],
 [11, 12, 13],
 [15, 16, 17],
 [16, 17, 18],
 [17, 18, 19],
 [18, 19, 20],
 [19, 20, 21],
 [20, 21, 22],
 [21, 22, 23],
 [22, 23, 24],
 [23, 24, 25],
 [27, 28, 29],
 [28, 29, 30],
 [29, 30, 31],
 [30, 31, 32],
 [31, 32, 33],
 [32, 33, 34],
 [36, 37, 38],
 [37, 38, 39],
 [38, 39, 40],
 [39, 40, 41],
 [40, 41, 42],
 [41, 42, 43],
 [42, 43, 44],
 [47, 48, 49],
 [48, 49, 50],
 [49, 50, 51],
 [50, 51, 52],
 [51, 52, 53],
 [52, 53, 54],
 [56, 57, 58],
 [57, 58, 59],
 [58, 59, 60],
 [59, 60, 61],
 [60, 61, 62],
 [61, 62, 63],
 [62, 63, 64],
 [67, 68, 69],
 [68, 69, 70],
 [69, 70, 71],
 [70, 71, 72],
 [71, 72, 73],
 [72, 73, 74],
 [76, 77, 78],
 [77, 78, 79],
 [78, 79, 80],
 [79, 80, 81],
 [80, 81, 82],
 [81, 82, 83],
 [82, 83, 84],
 [83, 84, 85],
 [84, 85, 86],
 [88, 89, 90],
 [89, 90, 91],
 [90, 91, 92],
 [91, 92, 93],
 [92, 93, 94],
 [93, 94, 95],
 [97, 9

In [10]:
images.columns

Index(['directory', 'filename', 'size', 'frame'], dtype='object')

In [11]:
#use following to generate triplets

In [75]:
rel_paths = []
seq = images.loc[seq_no]
for triplet in triplets:
    files = []
    for num in triplet:
        f, d =seq.loc[seq['frame']==num, ['directory' , 'filename']].iloc[0, :]
        files.append(join(f, d))
    rel_paths.append(files)
print("{0} triplets selected".format(len(rel_paths)))

164 triplets selected


In [195]:
import cv2
import numpy as np
from cv2 import calcOpticalFlowFarneback as sw_farneback_optical_flow


In [196]:
def write_triplet(paths):
    for i in range(len(paths)):
        im.append(read_source_image(join(orig_images_dir, paths[i])))
#     scaling to 0 --> 255
    scaled_ims = im-(np.min(im))
    scaled_ims /= np.max(scaled_ims) / 255.0
    scaled_ims = 255 - scaled_ims.astype(np.uint8)
    for i in range(len(scaled_ims)):
        cv2.imwrite('{0}_orig.png'.format(i), scaled_ims[i])
    

In [226]:
def warp_flow(img, flow):
    h, w = flow.shape[:2]
    flow = -flow
    flow[:,:,0] /=2.0
    flow[:,:,1] /=2.0
    flow[:,:,0] += np.arange(w)
    flow[:,:,1] += np.arange(h)[:,np.newaxis]
    res = cv2.remap(img, flow, None, cv2.INTER_CUBIC, borderMode =cv2.BORDER_REPLICATE )
    return res

In [228]:
for path in rel_paths[-2:-1]:
    write_triplet(path)

In [231]:
fr_1_gr = cv2.imread('0_orig.png')
fr_1_gr_ = cv2.cvtColor(fr_1_gr, cv2.COLOR_RGB2GRAY)
fr_2_gr = cv2.imread('2_orig.png')
fr_2_gr_ = cv2.cvtColor(fr_2_gr, cv2.COLOR_RGB2GRAY)
sw_flow = cv2.calcOpticalFlowFarneback(fr_1_gr_, fr_2_gr_, flow=None,
                                        pyr_scale=0.5, levels=1, winsize=15,
                                        iterations=2,
                                        poly_n=5, poly_sigma=1.1, flags=0)

In [241]:
def draw_flow(img, flow, step=22):
    h, w = img.shape[:2]
    y, x = np.mgrid[step/2:h:step, step/2:w:step].reshape(2,-1)
    x = x.astype(int)
    y = y.astype(int)
    fx, fy = flow[y,x].T
    lines = np.vstack([x, y, x+fx, y+fy]).T.reshape(-1, 2, 2)
    lines = np.int32(lines + 0.5)
    vis = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    cv2.polylines(vis, lines, 0, (0, 255, 0))
    for (x1, y1), (x2, y2) in lines:
        cv2.circle(vis, (x1, y1), 1, (0, 255, 0), -1)
    return vis

In [243]:
cv2.imwrite('1_flow.png',warp_flow(fr_1_gr, sw_flow))


True

In [242]:
draw_flow_ = draw_flow(fr_1_gr, sw_flow)

In [220]:
cv2.imwrite('draw_flow.png',draw_flow)

True